In [ ]:
import numpy as np
import matplotlib.pylab as plt

import lichen as lch

# 1-D histograms

In [ ]:
x = np.random.normal(5,1,10000)
h = lch.hist(x,bins=50)

In [ ]:
x = np.random.normal(5,1,10000)
wts = np.random.random(10000)
h = lch.hist(x,bins=50,weights=wts,range=(0,10))


# 2-D histograms

In [ ]:
mean = [0, 0]
cov = [[1, 0], [0, 2]]  # diagonal covariance
x, y = np.random.multivariate_normal(mean, cov, 50000).T

plt.figure()
h = lch.hist2d(x,y)


In [ ]:
plt.figure()
h = lch.hist2d(x,y,xbins=50,ybins=50,cmap=plt.cm.plasma)


In [ ]:
plt.figure()
h = lch.hist2d(x,y,xbins=50,ybins=50,xrange=(-2,2),yrange=(-2,2),cmap=plt.cm.gray)


# Generate and fit data using scipy.optimize

In [ ]:
import scipy.stats as stats

from lichen.fit import Parameter,get_numbers,reset_parameters,pois,errfunc
from lichen.fit import pretty_print_parameters,get_values_and_bounds,fit_emlm

## Generate some test data: peak on background

In [ ]:
npeak = 200
nbkg = 400

data = np.random.normal(5,0.2,npeak).tolist()
data += (4*np.random.random(nbkg) + 3).tolist()

plt.figure()
lch.hist(data);

## Fit using unbinned extended maximum likelihood

### Create a dictionary of lichen Parameters

Note that the dictionary entries have a key value that maps on to the PDF.

Each of the PDF entries has it's own dictionary of lichen ```Parameter``` objects.

The ```Parameter``` class just takes a starting value and a range to restrict the value during the fitting process. 

If you don't want to restrict the value, just put ```None``` for the range. 

In [ ]:
pars = {}
pars["peak"] = {"number":Parameter(300,(0,500)), "mean":Parameter(4.5,(4.0,6.0)), "sigma":Parameter(0.25,(0.10,0.5))}
pars["bkg"] = {"number":Parameter(300,(0,500))}

## Define your PDF functions

Your PDF functions must be normalized over the range you're fitting.

Note that I've defined these with a ```frange``` kwd for "function range".

In [ ]:
################################################################################
def peak(pars, x, frange=None):

    mean = pars["peak"]["mean"].value
    sigma = pars["peak"]["sigma"].value

    pdfvals = stats.norm(mean,sigma).pdf(x)

    return pdfvals
################################################################################

################################################################################
def background(x, frange=None):

    # Flat
    height = 1.0/(frange[1] - frange[0])

    pdfvals = height*np.ones(len(x))

    return pdfvals
################################################################################

################################################################################
def pdf(pars,x,frange=None):

    frange = (3,7) # Hard coded for now, but will change in the future
    
    npeak = pars["peak"]["number"].value
    nbkg = pars["bkg"]["number"].value

    ntot = float(npeak + nbkg)

    bkg = background(x,frange=frange)
    p0 = peak(pars,x,frange=frange)


    totpdf = (npeak/ntot)*p0 + (nbkg/ntot)*bkg

    return totpdf
################################################################################



## Run the fit!

The initial values of the parameters will be whatever the ```Parameter``` objects were initialized to.

*Warning! If you re-run the cell below, you will want to re-run the cell that defined the ```pars``` dictionary if you want to get back to the starting values. The parameters were updated during the fit process.*

In [ ]:
initvals,finalvals = fit_emlm(pdf,pars,[data],verbose=False)

print("Done with fit!")
print()

pretty_print_parameters(pars)


## Plot the results on the data

In [ ]:
# Plot the data
plt.figure()
lch.hist(data,bins=50);


# Plot the PDFs
xpts = np.linspace(3,7,1000)
binwidth=(4/50)

ysig = pars['peak']['number'].value * peak(pars,xpts) * binwidth
plt.plot(xpts,ysig,linewidth=3,label='signal')

ybkg = pars['bkg']['number'].value * background(xpts,frange=(3,7)) * binwidth
plt.plot(xpts,ybkg,'--',linewidth=3,label='background')

ytot = ysig + ybkg
plt.plot(xpts,ytot,linewidth=3,label='total pdf')

plt.legend()